In [14]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [16]:
# 1 request로 html 저장하기, 삼성전자로 시작
## 2 soup객체에 beautifulsoup 담기

url = "https://finance.naver.com/item/frgn.nhn?code=005930&page=1"
req = requests.get(url)
html = req.text
print(html)
soup = bs(html,'html.parser')

<!--  global include -->

	
	
<html lang='ko'>
<head>


	
		<title>삼성전자 : 네이버 금융</title>
	
	





	
	
		<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	


<meta http-equiv="Content-Script-Type" content="text/javascript">
<meta http-equiv="Content-Style-Type" content="text/css">





	
		<meta property="og:title" content="네이버 금융"/>
		<meta property="og:image" content="https://ssl.pstatic.net/static/m/stock/im/2016/08/og_stock-200.png"/>
		<meta property="og:url" content="https://finance.naver.com"/>
		<meta property="og:description" content="국내 해외 증시 지수, 시장지표, 펀드, 뉴스, 증권사 리서치 등 제공"/>
    
    

<meta property="og:type" content="article"/>
<meta property="og:article:thumbnailUrl" content=""/>
<meta property="og:article:author" content="네이버금융"/>
<meta property="og:article:author:url" content="http://FINANCE.NAVER.COM"/>



<link rel='stylesheet' type='text/css' href='/css/finance_header.css?20201217181526'>

	
	
	
	
		
		<link rel="stylesheet" type="text/css" href="

In [136]:
# 1) 상장주식 종목 코드 구하기
## KRX 홈페이지에서 상장법인목록 엑셀 파일 내려받은 다음 추출하기

df = pd.read_excel('C:/Users/PS42/Desktop/PBR시가총액크롤링/상장법인목록전체.xlsx')
com_name = df["회사명"]
com_code = df["종목코드"]

In [159]:
# 2) url iterate 해서 데이터 구하기

## code_list 만들기(iterater 만들기 위함)

code_list = []
for i in range(len(com_code)):
    code_list.append(com_code[i])

## pbr_list 만들기

pbr_list = []

for code in code_list[1:2]:
    
    html = requests.get('https://finance.naver.com/item/frgn.nhn?code=%s&page=1'%code) 
    
    table = pd.read_html(html.text)
    
    a = table[7][1][2]
    num = a.replace(" ","").split("l")[1].replace("원","").replace(",","")
    current_price = float(num)
    
    bps = table[2]["종가"].iloc[0][0]
    
    pbr = current_price / bps
    
    pbr_list.append(pbr)

    

ValueError: could not convert string to float: 'N/A'

### 필요한 데이터 다 구했고, 이걸로 최종 마무리!!!
###### * 위에는 beautifulsoup, request, pandas 활용한 데이터 크롤링 시도.필요한 데이터가 이미 정제된 파일이 있어서, 그걸 가져다 쓰기로 결정

1) PBR 데이터 가져와서 하위 20% 나열

2) 시가총액 데이터 두 개(KOSPI, KOSDAQ) 가져와서 합친 뒤 하위 20% 나열

3) 겹치는 종목 발굴!

In [1]:
# 1) PBR 데이터 가져오기
import pandas as pd

In [77]:
# 데이터 불러오기
df_pbr = pd.read_excel('C:/Users/PS42/Desktop/PBR시가총액크롤링/상장기업PBR.xlsx')

In [78]:
# PBR 기재 안 된 종목들 없애기
none_list = []

for i in range(len(df_pbr)):
    if df_pbr["PBR"][i] == "-":
        none_list.append(i)
        
df_pbr_fix = df_pbr.drop(df_pbr.index[none_list])
df_pbr_fix.sort_values(by="PBR",ascending=True)
df_pbr_fix = df_pbr_fix.reset_index()

In [80]:
## 행 기준으로 하위 20%인 DATA들 추출 loc 메소드 INDEX 기준으로 추출(427 -> 종목 중 20% 정도 수치)
df_pbr_fix = df_pbr_fix.sort_values(by="PBR",ascending=True)
df_pbr_fix = df_pbr_fix.reset_index()
df_pbr_fix_rank20 = df_pbr_fix.loc[:427]
df_pbr_fix_rank20

In [192]:
# 2) 시가총액 데이터 불러와서 합친 뒤 정렬
## 데이터 합치기
df_total_kospi = pd.read_excel('C:/Users/PS42/Desktop/PBR시가총액크롤링/KOSPI시가총액.xlsx')
df_total_kosdaq = pd.read_excel('C:/Users/PS42/Desktop/PBR시가총액크롤링/KOSDAQ시가총액.xlsx')
df_total_result = pd.concat([df_total_kospi,df_total_kosdaq], ignore_index = True)

In [194]:
## 시가총액이 문자형으로 되어있어, 숫자형으로 바꿔주고 column 추가
total_str = list(df_total_result["시가총액"])

total_strtonum = []
for string in total_str:
    string = string.replace(",","")
    total_strtonum.append(string)

df_total_result["시가총액숫자"] = total_strtonum
df_total_result["시가총액숫자"] = df_total_result["시가총액숫자"].astype(float)
df_total_result = df_total_result.sort_values("시가총액숫자",ascending = True).reset_index()

In [212]:
## 시가총액 하위 20% 최종 완성
df_total_result_rank20 = df_total_result.loc[0:len(df_total_result)*0.2]

In [219]:
# 3) 겹치는 종목 발굴하기

## 여기서는 list 인스턴트(?) 코드 활용 , final_stockpicking이 최종 pick된 list

pbr_stockpicking = list(df_pbr_fix_rank20["종목명"])
total_stockpicking = list(df_total_result_rank20["종목명"])

final_stockpicking = [x for x in pbr_stockpicking if x in total_stockpicking]
final_stockpicking

In [235]:
## df_final에 최종 선정된 종목들의 데이터 표를 담기

df_pbr_fix_rank20 = df_pbr_fix_rank20.set_index("종목명").iloc[:,2:]
df_final = df_pbr_fix_rank20.loc[final_stockpicking]

In [260]:
## 추가 조건식 : PER <= 15 인 종목들만 골라내기

df_final_filter1 = df_final[df_final["PER"]!="-"]
df_final_filter1["PER"] = df_final_filter1["PER"].astype(float)
df_final_filter1[df_final_filter1["PER"]<=15]

위 표 중에 차트 정배열인거 골라서 동일 비중으로 투자할 것!

최종 조건 : 소형주, 저 PBR, 저 PER(<=15), 정배열
거래량 너무 없는 것과 동전주는 피하고

원일특강, 아즈텍WB, 원림, 와토스코리아, 화신정공